## III c

In [54]:
class Phi(nn.Module):
    """ Trivial implementation of Phi"""
    def forward(self,inputs):
        return inputs

In [47]:
import torch 
import torch.nn as nn

class F(nn.Module):
    def __init__(self,phi_in:int,phi:nn.Module): 
        """Creates a new instance of F.
        Parameters
        - - - - - -
        phi_in : int
        The input dimension of phi, i.e., the expected
        shape is [batch_size, phi_in]
        phi : nn.Module
        PyTorch module which implements a neural network block "Phi"
        """
        super().__init__()
        self.phi = phi
        self.phi_in = phi_in
        self.xb_size = phi_in[1]
        
    def forward(self, inputs):
        xa,xb=inputs.split([inputs.shape[1]-self.xb_size,self.xb_size],dim=1)
        ya = xa + self.phi(xb)
        yb = xb
        return torch.cat((ya,yb),axis=1)

    def inverse(self, inputs):
        ya,yb=inputs.split([inputs.shape[1]-self.xb_size,self.xb_size],dim=1)
        xb=yb
        xa=ya-self.phi(yb)
        return torch.cat((xa,xb),axis=1)


In [55]:
import unittest

def test_phi(x):
    return x

class TestF(unittest.TestCase):
    def setUp(self):
        phi_in = (3,2) # batch_size = 3
        phi_module=Phi()
        self.model=F(phi_in,phi_module)
        xa = torch.tensor(((1,2),(1,2),(1,2)))
        xb = torch.tensor(((3,4),(3,4),(3,4)))
        self.x = torch.cat((xa,xb),axis=1)
        ya = torch.tensor(((4,6),(4,6),(4,6)))
        yb = torch.tensor(((3,4),(3,4),(3,4)))
        self.y= torch.cat((ya,yb),axis=1)
        
    def test_forward(self):
        y_pred = self.model(self.x)
        self.assertTrue(torch.equal(y_pred,self.y))
        
    def test_inverse(self):
        x_pred = self.model.inverse(self.y)
        self.assertTrue(torch.equal(x_pred,self.x))
            
unittest.main(argv=[''], verbosity=2, exit=False)

test_forward (__main__.TestF) ... ok
test_inverse (__main__.TestF) ... ok
test_forward (__main__.TestS) ... ok
test_inverse (__main__.TestS) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


## III d

In [50]:
import torch
import torch.nn as nn

class S(nn.Module):
    def __init__(self, n_A, n_B):
        """ Creates a new instance of S.
        Parameters
        - - - - - -
        n_A : int
        Number of components for x_A
        n_B : int
        Number of components for x_B
        """
        
        super().__init__()
        self.n_A=n_A
        self.n_B=n_B
    
    def forward(self, inputs):
        xa,xb=inputs.split([self.n_A,self.n_B],dim=1)
        return torch.cat((xb,xa),dim=1)
    
    def inverse(self, inputs):
        xb,xa=inputs.split([self.n_B,self.n_A],dim=1)
        return torch.cat((xa,xb),dim=1)


In [51]:
import unittest

class TestS(unittest.TestCase):
    def setUp(self):
        self.model=S(2,2)
        # batch size 4
        xa = torch.tensor(((1,2),(1,2),(1,2),(1,2)))
        xb = torch.tensor(((3,4),(3,4),(3,4),(3,4)))
        self.forward=torch.cat(tensors=(xa,xb),dim=1)
        self.backward=torch.cat(tensors=(xb,xa),dim=1)
        
    def test_forward(self):
        y_pred = self.model(self.forward)
        self.assertTrue(torch.equal(y_pred,self.backward))
        
    def test_inverse(self):
        y_pred = self.model.inverse(self.backward)
        self.assertTrue(torch.equal(y_pred,self.forward))
                    
unittest.main(argv=[''], verbosity=2, exit=False)

test_forward (__main__.TestF) ... ok
test_inverse (__main__.TestF) ... ok
test_forward (__main__.TestS) ... ok
test_inverse (__main__.TestS) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.012s

OK


## III e 

In [64]:
class G(nn.Module):
    def __init__(self,n_A,n_B):
        """ Creates a new instance of G.
        Parameters
        - - - - - -
        n_A : int
        Number of components for x_A
        n_B : int
        Number of components for x_B
        """
        
        super().__init__()
        self.n_A=n_A
        self.n_B=n_B
        phi_module = Phi()
        
        self.stack=nn.Sequential(
        F(phi_in=(4,2),phi=phi_module),
        S(2,2),
        F(phi_in=(4,2),phi=phi_module),
        S(2,2),
        F(phi_in=(4,2),phi=phi_module))
        
    def forward(self,inputs):
        return(self.stack(inputs))
    def inverse(self,inputs):
        pass
    

In [69]:
import unittest

class TestG(unittest.TestCase):
    def setUp(self):
        self.model=G(2,2)
        # batch size 4
        xa = torch.tensor(((1,2),(1,2),(1,2),(1,2)))
        xb = torch.tensor(((3,4),(3,4),(3,4),(3,4)))
        ## calculated this solution by hand
        ya = torch.tensor(((11,16),(11,16),(11,16),(11,16)))
        yb = torch.tensor(((7,10),(7,10),(7,10),(7,10))) 
        self.input=torch.cat(tensors=(xa,xb),dim=1)
        self.solution=torch.cat(tensors=(ya,yb),dim=1)
        
    def test_forward(self):
        y_pred = self.model(self.input)
        self.assertTrue(torch.equal(y_pred,self.solution))
        
    def test_inverse(self):
        y_pred = self.model.inverse(self.solution)
        self.assertTrue(torch.equal(y_pred,self.input))
                    
unittest.main(argv=[''], verbosity=2, exit=False)

test_forward (__main__.TestF) ... ok
test_inverse (__main__.TestF) ... ok
test_forward (__main__.TestG) ... ok
test_inverse (__main__.TestG) ... ERROR
test_forward (__main__.TestS) ... ok
test_inverse (__main__.TestS) ... ok

ERROR: test_inverse (__main__.TestG)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-69-c6cb303d30b1>", line 21, in test_inverse
    self.assertTrue(torch.equal(y_pred,self.input))
TypeError: equal(): argument 'input' (position 1) must be Tensor, not NoneType

----------------------------------------------------------------------
Ran 6 tests in 0.019s

FAILED (errors=1)
